---

<div style="display: flex; align-items: center; justify-content: space-between;">
  <img src="../../.images/PNAV-logo.png" alt="Logo del PNAV" style="width: auto; max-height: 100px;">
  <img src="../../.images/MITECO-logo_background.png" alt="Logo del MITECO" style="width: auto; max-height: 100px;">
</div>

---

# Implementación de CodeCarbon
A continuación, se mostrará el proceso de cómo realizar las estimaciones sobre el consumo energético utilizando la libería de  `codecarbon` desde un notebook en Azure.

Para más información, consultar el [github](hhttps://github.com/mlco2/codecarbon/blob/master/README.md) de la herramienta.

⚠️ Este material es un notebook de demostración **orientativo** sobre la metodología que se debe seguir, no se debe replicar el desarrollo del caso de uso.

## 1. Instalación de librerías y paquetes

In [1]:
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install codecarbon

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.4.0
    Uninstalling termcolor-2.4.0:
      Successfully uninstalled termcolor-2.4.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


## 2. Definición del Power Usage Effectiveness (PUE)
En función de la región en la que se el usuario cree las instancias de computación, el PUE variará. A continuación, se muestra una tabla con los PUE por ubicación:

<div style="text-align: center;">
  <table>
    <tr>
      <th>Geography</th>
      <th>PUE</th>
    </tr>
    <tr>
      <td>Arizona</td>
      <td>1.18</td>
    </tr>
    <tr>
      <td>Illinois</td>
      <td>1.35</td>
    </tr>
    <tr>
      <td>Iowa</td>
      <td>1.16</td>
    </tr>
    <tr>
      <td>Texas</td>
      <td>1.28</td>
    </tr>
    <tr>
      <td>Virginia</td>
      <td>1.14</td>
    </tr>
    <tr>
      <td>Washington</td>
      <td>1.15</td>
    </tr>
    <tr>
      <td>Wyoming</td>
      <td>1.11</td>
    </tr>
    <tr>
      <td>Singapore</td>
      <td>1.34</td>
    </tr>
    <tr>
      <td>Ireland</td>
      <td>1.19</td>
    </tr>
    <tr>
      <td>Netherlands</td>
      <td>1.14</td>
    </tr>
    <tr>
      <td>Sweden</td>
      <td>1.16</td>
    </tr>
    <tr>
      <td>Spain</td>
      <td>1.12</td>
    </tr>
  </table>
</div>

Si el PUE utilizado no se encuentra en esta tabla, se deberá justificar.

Para más información, consultar los siguientes enlaces:
- https://news.microsoft.com/es-es/2024/06/11/microsoft-opens-its-first-cloud-region-in-spain-to-accelerate-the-development-of-the-ai-economy/
- https://datacenters.microsoft.com/sustainability/efficiency/

In [11]:
pue = 1.12

## 3. Selección del Cloud Provider y de la región
De momento, CodeCarbon no registra las regiones de Azure. Para poder realizar una aproximación, el usuario deberá indicar que el proveedor cloud es Google Cloud (`gcp`) y la región más cercana a la que esté utilizando en la instancia. Por ejemplo, si la instancia está creada en España, la región será `europe-southwest1`. Se pueden consultar las regiones disponibles en el [siguiente enlace](https://github.com/mlco2/codecarbon/blob/master/codecarbon/data/cloud/impact.csv) 

In [25]:
country_iso_code = 'ESP'
region='ESP'
cloud_provider='gcp'
cloud_region='europe-southwest1'
country_2letter_iso_code = 'ES'

## 4. Otros parámetros
El usuario podrá modificar otros parámetros del tracker según vea conveniente. Un listado de todos los parámetros modificables está disponible en la [documentación de codecarbon](https://mlco2.github.io/codecarbon/parameters.html).

### 4.1. Input Parameters

| Parameter                  | Description                                                                                  |
|----------------------------|----------------------------------------------------------------------------------------------|
| `project_name`              | Name of the project, defaults to `codecarbon`.                                               |
| `experiment_id`             | Id of the experiment.                                                                         |
| `measure_power_secs`       | Interval (in seconds) to measure hardware power usage, defaults to 15.                       |
| `tracking_mode`             | `machine` measure the power consumptions of the entire machine (defaults), `process` try and isolate the tracked processes in isolation.                                 |
| `gpu_ids`                   | User-specified known GPU ids to track, defaults to None.                                      |
| `log_level`                 | Global codecarbon log level (by order of verbosity): `debug`, `info` (defaults), `warning`, `error`, or `critical`. |
| `co2_signal_api_token`      | API token for co2signal.com (requires sign-up for free beta).                                 |
| `pue`                       | PUE (Power Usage Effectiveness) of the data center where the experiment is being run.        |
| `default_cpu_power`         | Default CPU power consumption in watts, defaults to 42.5 (POWER_CONSTANT x CONSUMPTION_PERCENTAGE_CONSTANT). |
| `allow_multiple_runs`      | Boolean variable indicating if multiple instances of CodeCarbon on the same machine are allowed, defaults to False. |
| `country_iso_code`          | 3-letter ISO Code of the country where the experiment is being run. Available countries are listed in `global_energy_mix.json`. |
| `region`                    | Optional Name of the Province/State/City, where the infrastructure is hosted. Currently, supported only for US States and Canada (e.g., California or New York). |
| `cloud_provider`            | The cloud provider specified for estimating emissions intensity, defaults to None. See `impact.csv` for a list of cloud providers. |
| `cloud_region`              | The region of the cloud data center, defaults to None. See `impact.csv` for a list of cloud regions. |
| `country_2letter_iso_code` | For use with the Electricity Maps emissions API. See Electricity Maps zones for a list of codes and their locations. |

### 4.2. Output Parameters
| Parameter                  | Description                                                                                  |
|----------------------------|----------------------------------------------------------------------------------------------|
| `save_to_file`              | Boolean variable indicating if the emission artifacts should be logged to a CSV file, defaults to True. |
| `output_dir`                | Directory path to which the experiment details are logged, defaults to current directory.    |
| `output_file`               | Name of output CSV file, defaults to `emissions.csv`.                                         |
| `on_csv_write`              | When calling `tracker.flush()` manually choose if:                                            |
|                            | - update the existing `run_id` row (erasing former data)                                     |
|                            | - append add a new row to CSV file (defaults).                                               |
| `save_to_api`               | Boolean variable indicating if emissions artifacts should be logged to the CodeCarbon API, defaults to False. |
| `api_endpoint`              | Optional URL of CodeCarbon API endpoint for sending emissions data, defaults to `https://api.codecarbon.io`. |
| `api_key`                   | API key for the CodeCarbon API (mandatory to use this API!).                                 |
| `api_call_interval`         | Number of measurements between API calls (defaults to 8):                                    |
|                            | -1 : call API on `flush()` and at the end                                                    |
|                            | 1 : at every measure                                                                          |
|                            | 2 : at every 2 measure, and so on                                                           |
| `save_to_logger`            | Boolean variable indicating if the emission artifacts should be written to a dedicated logger, defaults to False. |
| `logging_logger`            | `LoggerOutput` object encapsulating a `logging.Logger` or a Google Cloud logger.             |
| `logger_preamble`           | String to systematically include in the logger’s messages (defaults to “”).                  |
| `save_to_prometheus`        | Boolean variable indicating if the emission artifacts should be written to a Prometheus server, defaults to False. |
| `prometheus_url`            | URL of the Prometheus server.                                                                |
| `save_to_logfire`           | Boolean variable indicating if the emission artifacts should be written to a LogFire server, defaults to False. |
| `output_handlers`           | List of output handlers to use for saving the emissions data, defaults to `[]`.               |


In [39]:
# Para este caso, únicamnete se definirá el nombre del fichero de salida. Si el usuario lo desea, puede añadir otros.
output_file = 'test_emissions.csv'

## 5. Creación del Tracker
El usuario deberá crear una instancia del Tracker con los parámetros seleccionados previamente. Para evitar errores y posibles incompatibilidades, se utilizará el Tracker Offline.

In [40]:
from codecarbon import OfflineEmissionsTracker
tracker = OfflineEmissionsTracker(
        country_iso_code=country_iso_code,
        region=region,
        cloud_provider=cloud_provider,
        cloud_region=cloud_region,
        country_2letter_iso_code=country_2letter_iso_code,
        pue=pue,
        output_file = output_file
        )

[codecarbon INFO @ 11:47:52] offline tracker init
[codecarbon INFO @ 11:47:52] [setup] RAM Tracking...
[codecarbon INFO @ 11:47:52] [setup] CPU Tracking...
[codecarbon WARNING @ 11:47:52] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon WARNING @ 11:48:35] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1265U but we don't know it. Please contact us.
[codecarbon INFO @ 11:48:35] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1265U
[codecarbon INFO @ 11:48:35] [setup] GPU Tracking...
[codecarbon INFO @ 11:48:35] No GPU found.
[codecarbon INFO @ 11:48:35] >>> Tracker's metadata:
[codecarbon INFO @ 11:48:35]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 11:48:35]   Python version: 3.10.11
[codecarbon INFO @ 11:48:35]   CodeCarbon version: 2.8.2
[codecarbon INFO @ 11:48:35]   Available RAM : 15.441 GB
[codecarbon INFO @ 11:48:35]   CPU count: 12
[c

## 6. Medición del consumo
Para realizar las mediciones, se deberán utilizar las funciones `tracker.start()` y `tracker.stop()` de la siguiente forma.
Una finalizado el flujo se generará un fichero con las mediciones en el directorio indicado en el tracker. Si no se indica ningún directorio, por 

In [41]:
# Inicio del proceso de medición
tracker.start()

# Tu código aquí

# Fin del proceso de medición
emissions = tracker.stop()

# Accede a las emisiones calculadas
print(f"Emissions: {emissions} kg CO₂eq")

[codecarbon INFO @ 11:49:39] Energy consumed for RAM : 0.000000 kWh. RAM Power : 5.790398597717285 W
[codecarbon INFO @ 11:49:39] Energy consumed for all CPUs : 0.000001 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:49:39] 0.000001 kWh of electricity used since the beginning.


Emissions: 1.3336321996364742e-07 kg CO₂eq


c:\Users\mario.b.garcia\AppData\Local\Programs\Python\Python310\lib\site-packages\codecarbon\output_methods\file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
